In [7]:
#Another way of finding the class I want. And assigning it the name whatever_class
Project_class=onto.search(iri='*Project*')
print(Project_class)

[gist.Project]


In [ ]:
print('\nAll object properties\n'+'-'*20)
for x in properties: 
    print(x)
print()

In [24]:
rel = properties[-1]
print(rel)
print(rel.__dict__)
print()

gist.viableRange
{'namespace': get_ontology("https://ontologies.semanticarts.com/o/gistCore#").get_namespace("https://ontologies.semanticarts.com/gist/"), 'is_a': [owl.ObjectProperty], 'storid': 599, '_name': 'viableRange', '_equivalent_to': None, '__module__': 'owlready2.entity', '__doc__': None, '_domain': None, '_range': None, '_property_chain': None, '_inverse_property': None, '_python_name': 'viableRange', '_class_property_type': [], '_class_property_some': True, '_class_property_only': False, '_class_property_relation': False, '_inverse_storid': 0}



In [26]:
"""
The .class_property_type attribute of Properties allows to indicate how to handle class properties:
    “some”: handle class properties as existential restrictions (i.e. SOME restrictions and VALUES restrictions).
    “only”: handle class properties as universal restrictions (i.e. ONLY restrictions).
    “relation”: handle class properties as relations (i.e. simple RDF triple, as in Linked Data).
"""
print('class_property_some:', rel._class_property_some)
print('class_property_only:', rel._class_property_only)
print('class_property_relation:', rel._class_property_relation)

class_property_some: True
class_property_only: False
class_property_relation: False


In [25]:
print('name(string):', rel.name)
print('module_type:', rel.__module__)
print('is_a:', rel.is_a)
print()

name(string): viableRange
module_type: owlready2.entity
is_a: [owl.ObjectProperty]



In [32]:
#Interested in the relation 'affects'
governedBy_property=onto.search(iri='*governedBy*')

In [33]:
print(governedBy_property[0])
print('class_property_some:', governedBy_property[0]._class_property_some)
print('class_property_only:', governedBy_property[0]._class_property_only)
print('class_property_relation:', governedBy_property[0]._class_property_relation)

gist.governedBy
class_property_some: True
class_property_only: False
class_property_relation: False


In [ ]:
# Retrieve all disjoint sets
disjoints = list(onto.disjoints())
print('\nAll disjoints\n'+'-'*20)
for x in disjoints: 
    # NOTE: Retreive all disjoint individual entity objects, stored in the dict
    print(x)
    print(x.__dict__)
    print()